# **次の実践課題: 多様なデータソースの利用と変化検出**

## **課題の概要**

ここでは、**SAR (合成開口レーダー) データ**という、これまで扱ってきた光学データ（Landsat）とは全く異なるデータソースを利用し、特定のイベント（ここでは**洪水**）前後の変化を検出する課題に挑戦します。

SARデータは雲や夜間の影響を受けないため、災害時のモニタリングに不可欠です。

---

## **ステップ1: SAR データの取得とフィルタリング**

**<span style='color:red'>SAR データとは</span>**

光学衛星（Landsat, Sentinel-2など）が光の反射を利用するのに対し、SAR 衛星（Sentinel-1など）はマイクロ波を地表に照射し、その散乱波を観測します。これにより、雲や雨、夜間の影響を受けずにデータを取得できます。

### **課題 1-1: Sentinel-1 コレクションのフィルタリング**

ヨーロッパ宇宙機関 (ESA) の **Sentinel-1 GRD**（Ground Range Detected）データセットを使用します。

以下の条件で Sentinel-1 コレクションをフィルタリングしてください。

1. **データセット**: `COPERNICUS/S1_GRD`
2. **期間**:
   - **洪水前**: 2024年7月1日 から 2024年7月30日
   - **洪水後**: 2024年8月1日 から 2024年8月30日
4. **領域**: 以前使用した琵琶湖周辺の領域 (`ee.Geometry`)。
5. **モード**: 地表を広範囲に観測する IW (Interferometric Wide Swath) モードに限定します。
6. **偏波 (Polarization)**: 垂直送信・垂直受信の $\text{VV}$ 偏波を持つ画像に限定します。

#### **Guiding Question**: 上記の条件（特に $\text{mode='IW'}$ と $\text{POLARIZATION='VV'}$）を適用し、**洪水前**のコレクションを s1_pre_collection に、**洪水後**のコレクションを s1_post_collection に格納するには、どのようにコードを記述すればよいでしょうか？

```python
# ヒント: .filter(ee.Filter.listContains(プロパティ名, 値)) を使用してリストプロパティをフィルタリングします。
# 例: ee.Filter.listContains('バンド名リスト', 'VV')

# 領域は以前定義した 'region' 変数を使用してください
```

---
---
**<span style='color:red'>(Wrong!)</span>**

```python
## 1. 地域の定義 (琵琶湖の中心点)
region = ee.Geometry.Point(136.08, 35.05)

s1_pre_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.filter.date('2024-07-01', '2024-07-30')) # 括弧の閉じ方に構文エラー
    .filter(ee.Filter.bounds(region))
    .select('VV')
)

s1_post_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.filter.date('2024-08-01', '2024-08-30'))  # 括弧の閉じ方に構文エラー
    .filter(ee.Filter.bounds(region))
    .select('VV')
) 
```
ナイスアテンプトです！データの種類やモードの指定は完璧です。

しかし、フィルタリングのメソッドの呼び出しと括弧の閉じ方に構文エラーがあります。`.filter(ee.Filter.date(...))` や `.filter(ee.Filter.bounds(...))` を使う代わりに、コレクションに直接メソッドをチェーン接続するのが最もクリーンな方法です。
---

---
---
**<span style='color:blue'>(Correct!)</span>**

### **課題 1-1 の解答**
---
**コードの修正 (正しいフィルタリング構文)**

以下の通り、`filterDate()` と `filterBounds()` を直接チェーン接続し、最後の括弧を閉じます。
```python
region = ee.Geometry.Point(136.08, 35.05)

s1_pre_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # 修正点: filterDate() と filterBounds() を直接チェーン接続
    .filterDate('2024-07-01', '2024-07-30')
    .filterBounds(region)
    .select('VV')
)

s1_post_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # 修正点: filterDate() と filterBounds() を直接チェーン接続
    .filterDate('2024-08-01', '2024-08-30')
    .filterBounds(region)
    .select('VV')
)
```
---

## **ステップ2: データの確認と合成**

### **課題 2-1: 画像数の確認**

SARデータは光学データと異なり、頻繁に取得されます。フィルタリングが意図通りに行われたか確認するため、それぞれのコレクションに含まれる画像の数を取得しましょう。

#### **Guiding Question**: 各コレクションに含まれる画像の数を確認するために、それぞれのコレクション変数に対してどのメソッドを使用し、結果を Python 環境に出力する必要がありますか？

**<span style='color:red'>Answer:</span>**
`getInfo()` は、コレクションの全てのメタデータを取得する一般的なメソッドですが、コレクションの要素の数を効率的に取得するためには、専用のメソッド `.size()` を使います。

### **課題 2-1 の解答**
---
```python
# 洪水前の画像数を取得
pre_size = s1_pre_collection.size().getInfo()
print(f"洪水前の画像数: {pre_size}")

# 洪水後の画像数を取得
post_size = s1_post_collection.size().getInfo()
print(f"洪水後の画像数: {post_size}")
```
---

## **ステップ3: SAR 画像の合成とノイズ除去**

SAR 画像には特有のノイズ（**スペックル**）が含まれているため、分析に進む前に、ノイズを低減する必要があります。

### **課題 3-1: SAR 画像のノイズ低減と合成**

これまでの光学画像と同様に、コレクション内の複数の画像を統計的に統合することでノイズが大幅に低減されます。

#### **Guiding Question**: SAR 画像のノイズを低減し、それぞれのコレクション（`s1_pre_collection`, `s1_post_collection`）から単一の合成画像を作成し、それぞれ `sar_pre_composite` と `sar_post_composite` という変数に格納するには、どの還元 (Reducer) メソッドを使用すればよいでしょうか？

#### **課題 3-1 の解答** [1]
---
```python
# ヒント: 以前の課題で使用した合成メソッドと同じです。

sar_pre_composite = s1_pre_collection.median() # <ここにメソッドを記述>

sar_post_composite = s1_post_collection.median() # <ここにメソッドを記述>

```    
---
median() を使用することで、SAR 画像特有のノイズであるスペックルを効果的に低減し、分析に適した単一の合成画像を作成できました。

[1] 04_cloud_free.ipynb

## **ステップ4: SAR データによる変化検出**

SAR データにおいて、水面（洪水域）は**地表に吸収されず、照射角とは逆方向に反射される**ため、衛星に戻る信号（後方散乱）が**非常に弱く**なります。画像上では、水面は**暗い（低いピクセル値）**領域として現れます。

洪水前後の変化を検出するには、このピクセル値の急激な低下を捉える必要があります。最も一般的な方法は、比率 (Ratio) を計算することです。

### **課題 4-1: 洪水前後の比率計算**

洪水後 ($\text{Post}$) の画像を洪水前 ($\text{Pre}$) の画像で割ることで、どのピクセルで信号が減少したか（$\text{Ratio} < 1$）をハイライトできます。$$\text{Change Ratio} = \frac{\text{sar\_post\_composite}}{\text{sar\_pre\_composite}}$$

#### **Guiding Question**: 洪水後の合成画像 (`sar_post_composite`) を洪水前の合成画像 (`sar_pre_composite`) で割り、結果を `ratio_image` という変数に格納するには、どのバンド演算メソッドを使えばよいでしょうか？

#### **課題 4-1 の解答** [2]
---
```python
ratio_image = sar_post_composite.divide(sar_pre_composite)
```
---
これで、洪水による後方散乱の変化（信号強度の変化）を示す画像ができました。

[2] 05_cloud_free_add_ndvi.ipynb

## **ステップ5: 洪水域の抽出（閾値処理）**

`ratio_image` では、洪水の影響を受けた領域は**信号強度が急激に低下**しているため、値が $\mathbf{1}$ 未満になります。

### **課題 5-1: 洪水域の二値化**

洪水域を抽出するには、この比率画像に**閾値 (Threshold)を適用**し、洪水が起こった場所とそうでない場所を区別する必要があります。この処理を**二値化**と呼びます。  
一般的に、洪水域は**比率が $\mathbf{0.8}$ 未満**になったピクセルで識別できることが多いです。

#### **Guiding Question**: `ratio_image` に対して、**値が $\mathbf{0.8}$ 未満**のピクセルを $\mathbf{1}$、それ以外を $\mathbf{0}$ とする新しい二値画像 `flood_mask` を作成するには、どのバンド演算メソッドを使えばよいでしょうか？

#### **課題 5-1 の解答**
---
「$\mathbf{0.8}$ 未満」という比較演算には、`.lt()` (less than) メソッドを使用します。GEEでは、画像に対して比較演算子（`.lt()`, `.gt()`, `.eq()` など）を適用すると、条件を満たすピクセルは自動的に $\mathbf{1}$、満たさないピクセルは $\mathbf{0}$ となる**二値画像**が返されます。
```python
# ヒント: 比較演算子（.lt() や .gt()）を使い、その後に .rename() を使ってマスクに名前を付けます。

# 比率が0.8未満のピクセルを1 (洪水域)、それ以外を0とする
flood_mask = ratio_image.lt(0.8).rename('Flood_Mask')
```

---

## **ステップ6: 最終的な洪水マップの表示**

これで、洪水域 ($\text{1}$) と非洪水域 ($\text{0}$) を示すマスク画像（`flood_mask`）が完成しました。最終的な視覚化として、このマスクを洪水前の SAR 画像に重ねて表示し、実際に変化した場所をハイライトしましょう。

### **課題 6-1: マップへの追加**

洪水域を**青色**で表示し、その下層に洪水前の $\text{SAR}$ 画像を表示します。

#### **Guiding Question**: $\text{SAR}$ 画像はピクセル値の範囲が広いため、表示には適切な視覚化パラメータが必要です。以下のパラメータを参考に、最終的な洪水マップを作成し、地図に追加するにはどのようにコードを記述すればよいでしょうか？

- 洪水前のSAR画像 (`sar_pre_composite`) の視覚化:
    - $\text{min}$:$-25$
    - $\text{max}$: $0$
    - $\text{palette}$: $\text{'black', 'white'}$ (SARは通常、モノクロで表示されます)
- 洪水マスク (`flood_mask`) の視覚化:
    - $\text{min}$: $0$
    - $\text{max}$: $1$
    - $\text{palette}$: $\text{'00000000', '0000FF'}$ ($\text{0}$ は透明、$\text{1}$ は青色)
    
**(ヒント: 以前の課題のタイルURL取得ロジックと、Foliumへの追加ロジックを使用します)**

#### **課題 6-1 の解答** [3]
---
```python
# 3. タイルURLを取得
classification_map_id = classified_image.getMapId(classification_vis)
classification_tile_url = classification_map_id['tile_fetcher'].url_format

# Foliumへの追加
my_map = folium.Map(
        location=[latitude, longitude],   # 地図の中心座標
        zoom_start=10,                    # 初期ズームレベル (琵琶湖周辺)
        tiles=basemaps['Google Satellite Hybrid'] # 初期タイルを設定
    )
```

---
[3] 07_land_cover_class_20251123.ipynb

---
```python
sar_pre_vis_params = {
    'min' : -25,
    'max' : 0,
    'palette' : ['black', 'white']
}

sar_pre_composite_map_id = sar_pre_composite.getMapId(sar_pre_vis_params)
sar_pre_composite_tile_url = sar_pre_composite_map_id['tile_fetcher'].url_format

flood_mask_vis_params = {
    'min' : 0,
    'max' : 1,
    'palette' : ['00000000', '0000FF']
}

flood_mask_map_id = flood_mask.getMapId(flood_mask_vis_params)
flood_mask_tile_url = flood_mask_map_id['tile_fetcher'].url_format

my_map = folium.map(
    location = region,
    zoom_start = 10,
    tiles=basemaps['Google Satellite Hybrid']
)

folium.TileLayer(
    tiles = [sar_pre_composite_tile_url, flood_mask_tile_url],
    attr='Google Earth Engine',
    name='Flood Change',
    overlay=True,
    control=True
).add_to(my_map)

```
---
    
    
    
    

これは、SARデータを使った洪水変化検出のロジックとしてはほぼ完璧です！素晴らしいです。ただし、このコードにはいくつか小さな**構文上の修正**と、**ロジック上の修正**が必要です。

**<span style ='color:blue'>構文とロジックの修正点</span>**
---
1. **Foliumの初期化の修正**
`folium.map` は正しくは `folium.Map` です（大文字の 'M'）。また、以前の課題で `my_map` は既に定義されているはずですが、ここでは新しい地図を作成するとして修正します。
   
2. **Foliumへのタイルレイヤー追加の修正**
`folium.TileLayer` には、**一度に一つのタイル URL** しか渡せません。洪水前の画像と洪水マスクは、それぞれ別の `folium.TileLayer' として追加する必要があります。
   
3. **$\text{SAR}$ 視覚化パラメータの調整**
`sar_pre_vis_params` では、値が $\text{-25}$ から $\text{0}$ になっていますが、$\text{SAR}$ **画像のノイズ低減**を促すため、対数スケールではなく**リニアスケール**で表示するために、画像をデシベル（dB）からパワー（反射強度）に変換する処理を追加するのが一般的です。
   
しかし、ここではシンプルに、**洪水前の画像を背景レイヤー**として、**その上にマスクを重ねる**ロジックで修正します。

**<span style ='color:blue'>修正後の最終コード</span>**
---

以下のコードで、洪水前の $\text{SAR}$ 画像を背景に、変化した洪水域を青色でオーバーレイする地図が作成されます。

```python
# 1. 洪水前のSAR画像の視覚化パラメータ (背景用)
sar_pre_vis_params = {
    'min' : -25,
    'max' : 0,
    'palette' : ['black', 'white']
}
sar_pre_composite_map_id = sar_pre_composite.getMapId(sar_pre_vis_params)
sar_pre_composite_tile_url = sar_pre_composite_map_id['tile_fetcher'].url_format

# 2. 洪水マスクの視覚化パラメータ (オーバーレイ用)
flood_mask_vis_params = {
    'min' : 0,
    'max' : 1,
    # 0 (非洪水) は透明、1 (洪水) は青色
    'palette' : ['00000000', '0000FF'] 
}
flood_mask_map_id = flood_mask.getMapId(flood_mask_vis_params)
flood_mask_tile_url = flood_mask_map_id['tile_fetcher'].url_format

# 3. 地図の初期化 (folium.Mapに修正)
my_map = folium.Map(
    location = [35.05, 136.08], # [緯度, 経度]の順で修正
    zoom_start = 10
)

# 4. レイヤーの追加 (それぞれ独立したTileLayerとして追加)

# a. 洪水前のSAR画像を背景として追加 (不透明度1.0)
folium.TileLayer(
    tiles = sar_pre_composite_tile_url,
    attr='Google Earth Engine',
    name='01. Pre-Flood SAR Composite',
    overlay=True,
    control=True
).add_to(my_map)

# b. 洪水マスクを重ねて追加 (不透明度0.7など)
folium.TileLayer(
    tiles = flood_mask_tile_url,
    attr='Google Earth Engine',
    name='02. Flood Mask (Ratio < 0.8)',
    overlay=True,
    control=True
).add_to(my_map)

folium.LayerControl().add_to(my_map)
display(my_map)
```
---

#### **Guiding Question**: 上記の修正コードを実行し、表示された地図を確認してください。あなたが想定した洪水域（琵琶湖周辺の低地など）は、青色で強調表示されましたか？

***<span style='color:orange'>Complete Code</span>***
---

In [ ]:
import ee
import datetime

In [ ]:
ee.Authen

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
# 課題 1-1の解答
region = ee.Geometry.Point(136.08, 35.05)

s1_pre_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # 修正点: filterDate() と filterBounds() を直接チェーン接続
    .filterDate('2024-07-01', '2024-07-30')
    .filterBounds(region)
    .select('VV')
)

s1_post_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # 修正点: filterDate() と filterBounds() を直接チェーン接続
    .filterDate('2024-08-01', '2024-08-30')
    .filterBounds(region)
    .select('VV')
)

In [ ]:
# 課題 2-1 の解答
# 洪水前の画像数を取得
pre_size = s1_pre_collection.size().getInfo()
print(f"洪水前の画像数: {pre_size}")

# 洪水後の画像数を取得
post_size = s1_post_collection.size().getInfo()
print(f"洪水後の画像数: {post_size}")

In [ ]:
sar_pre_composite = s1_pre_collection.median()
sar_post_composite = s1_post_collection.median()

In [ ]:
ratio_image = sar_post_composite.divide(sar_pre_composite)

In [ ]:
# 比率が0.8未満のピクセルを1 (洪水域)、それ以外を0とする
flood_mask = ratio_image.lt(0.8).rename('Flood_Mask')

In [ ]:
import folium

In [ ]:
# 1. 洪水前のSAR画像の視覚化パラメータ (背景用)
sar_pre_vis_params = {
    'min' : -25,
    'max' : 0,
    'palette' : ['black', 'white']
}
sar_pre_composite_map_id = sar_pre_composite.getMapId(sar_pre_vis_params)
sar_pre_composite_tile_url = sar_pre_composite_map_id['tile_fetcher'].url_format

# 2. 洪水マスクの視覚化パラメータ (オーバーレイ用)
flood_mask_vis_params = {
    'min' : 0,
    'max' : 1,
    # 0 (非洪水) は透明、1 (洪水) は青色
    'palette' : ['00000000', '0000FF'] 
}
flood_mask_map_id = flood_mask.getMapId(flood_mask_vis_params)
flood_mask_tile_url = flood_mask_map_id['tile_fetcher'].url_format

# 3. 地図の初期化 (folium.Mapに修正)
my_map = folium.Map(
    location = [35.05, 136.08], # [緯度, 経度]の順で修正
    zoom_start = 10
)

# 4. レイヤーの追加 (それぞれ独立したTileLayerとして追加)

# a. 洪水前のSAR画像を背景として追加 (不透明度1.0)
folium.TileLayer(
    tiles = sar_pre_composite_tile_url,
    attr='Google Earth Engine',
    name='01. Pre-Flood SAR Composite',
    overlay=True,
    control=True
).add_to(my_map)

# b. 洪水マスクを重ねて追加 (不透明度0.7など)
folium.TileLayer(
    tiles = flood_mask_tile_url,
    attr='Google Earth Engine',
    name='02. Flood Mask (Ratio < 0.8)',
    overlay=True,
    control=True
).add_to(my_map)

folium.LayerControl().add_to(my_map)
display(my_map)

これで、あなたは Earth Engine の最も強力で実用的なスキルセットである多様なデータソースの利用と変化検出分析を習得しました。

**<span style='color:blue'>習得したスキルセットのまとめ</span>**
---

1. **SARデータの理解と利用**: 光学データ（Landsat）では不可能だった、雲や夜間を問わない観測データ（Sentinel-1）を利用できました。
2. **ノイズ低減**: $\text{SAR}$ 特有のノイズ（スペックル）を $\text{.median()}$ で低減しました。
3. **比率法による変化検出**: 洪水前後の画像比率 $\left( \frac{\text{Post}}{\text{Pre}} \right)$ を計算し、地表の状態の変化を定量化しました。
4. **閾値処理と二値化**: 適切な閾値 ($\text{0.8}$) を適用し、洪水域を二値マスクとして明確に抽出しました。